In [ ]:
%load_ext autoreload
%autoreload 2

# Exploration Potentiel solaire - global, et à l'échelle d'une école

In [ ]:
# Executer ci dessous ci besoin pour récupérer les données
# !extract-sample-data
# Et pour sauver une version markdown des notebooks, utiliser
# jupyter nbconvert wms_potentiel_solaire.ipynb --to markdown --output-dir=exports/

In [ ]:
from owslib.wms import WebMapService

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.affinity import translate
from shapely.ops import unary_union
from shapely import intersection
from pathlib import Path
import numpy as np

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
import rasterio
from io import BytesIO
from rasterio.plot import show
from rasterio import MemoryFile
from rasterio.plot import show

In [ ]:
import owslib
owslib.__version__

#### Récupération data d'une école

In [ ]:
DATA_FOLDER = Path("../data")

saint_denis_path = DATA_FOLDER / "saint_denis_reference_data.gpkg"

ecoles = gpd.read_file(saint_denis_path, layer="bdtopo_education").to_crs(2154)
batiments = gpd.read_file(saint_denis_path, layer="bdtopo_batiment").to_crs(2154)

ID = "SURFACTI0000000002555648"

batiments_ecole = batiments[batiments.cleabs_left__zone == ID]
boite = batiments_ecole.geometry.total_bounds
batiments_ecole.plot()

# 1. Potentiel solaire: utilisation d'un flux WMS

#### Definitions WMS

In [ ]:
# https://geoservices.ign.fr/services-web-experts-energies-renouvelables
# url = 'https://data.geopf.fr/annexes/ressources/wms-r/enr.xml'
url = "https://data.geopf.fr/wms-r/wms?SERVICE=WMS&"
wms = WebMapService(url, version='1.3.0')
layer= 'IRRADIATION.SOLAIRE'

In [ ]:
minx = -667916
maxx = 1113194
miny = 5012341
maxy = 6800125
espg = "3857"
Y = (maxy-miny)//2000
X = (maxx-minx)//2000
print(X,Y)

#### Acquisition of the image

In [ ]:
img = wms.getmap(layers = [layer], srs = 'EPSG:3857', bbox = [minx,miny,maxx,maxy] , size = (X, Y), format= 'image/geotiff',transparent=True,mode='32bit')

In [ ]:
with MemoryFile(img) as memfile:
    with memfile.open() as dataset:
        metas = dataset.meta
        show(dataset)

There are 4 bands but no single value

In [ ]:
metas

## __PROBLEME__

On a bien la France mais pas l'échelle! Le flux WMS contains a RGBA array mais ne contient pas une valeur en absolu.

Il faut trouver une autre source.

# 2. Potentiel solaire: Fichier statique

In [ ]:
import os
import rasterio.mask

#### Verification du fichier à l'échelle de la France

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","red"])


In [ ]:

tile = "/ENR_1-0_IRR-SOL_TIFF_WGS84G_FXX_2023-10-01/1_DONNEES_LIVRAISON/GlobalHorizontalIrradiation.tif"
path = DATA_FOLDER / tile
with rasterio.open("../data/"+str(path)) as img:
    print(img.crs)
    show(img,title="Representation de l'IRR en france metropolitaine, en kWh/m2/an",cmap=cmap)

#### Focus autour de l'école

In [ ]:
batiments_ecole2 = batiments_ecole.copy()
geo = batiments_ecole.to_crs(epsg=6933).buffer(2000)  ## buffer de 2km
batiments_ecole2["geometry"] = geo
batiments_ecole2 = batiments_ecole2.to_crs(epsg=4326)

#### Potentiel moyen autour de l'école

In [ ]:
zone = batiments_ecole2
zone.to_crs(epsg=4326)
tile = "/ENR_1-0_IRR-SOL_TIFF_WGS84G_FXX_2023-10-01/1_DONNEES_LIVRAISON/GlobalHorizontalIrradiation.tif"
path = DATA_FOLDER / tile
with rasterio.open("../data/"+str(path)) as img:
    print(img.crs)
    out_image, out_transform = rasterio.mask.mask(img, zone.geometry, crop=True)

out_image[np.isnan(out_image)] = 0
avg = np.mean(out_image[out_image > 100])
print("Ensoleillement moyen:",avg,"kWh/m2/an")

# Assemblage pour une école

#### Fonction grossiere pour estimer la surface utile d'une ecole

In [ ]:
def surface_utile(surfacetotale):
    if surfacetotale < 500:
        ratio = 0.4*surfacetotale/5000+0.2
        if surfacetotale > 100:
            return ratio*surfacetotale
    else:
        return surfacetotale*0.6
    return 0
        

#### Hypothese de rendement

In [ ]:
# hypothese
rendement = 0.1

#### Selection de la layer de la BDD Potentiel Solaire Toiture

In [ ]:
src = DATA_FOLDER / "saint_denis_reference_data.gpkg"
potentiel = gpd.read_file(src, layer="potentielsolaire_toitures")
potentiel_ecole = potentiel[potentiel.id.isin(batiments_ecole.cleabs_left__bat)]

#### Methodologie de calcul de potentiel par batiment

In [ ]:
potentiel_ecole = potentiel_ecole[["id",'st_areashape',"eq_pano","eq_surf",'surf_util','moyenne2','production',"geometry",'st_lengthshape']]
potentiel_ecole["surf_utile_calculee"] = potentiel_ecole['st_areashape'].apply(lambda x: surface_utile(x) )
potentiel_ecole["prod_calculee"] = avg*potentiel_ecole["surf_utile_calculee"]*rendement
potentiel_ecole["fit"] = potentiel_ecole.prod_calculee / potentiel_ecole.production
potentiel_ecole["IRR"] = avg 
potentiel_ecole[["id",'surf_util',"surf_utile_calculee","moyenne2",'IRR','production','prod_calculee',"fit"]]

#### Aggregation des valeurs à l'échelle de l'école

In [ ]:
total_ecole         = potentiel_ecole.production.sum()
total_ecole_calcule = potentiel_ecole.prod_calculee.sum()
print("# Verification des potentiels pour l'école:",ID,"\n")
print("* BDD Potentiel solaire\t",int(total_ecole),"kWh/an")
print("* Estimation ordre 0\t",int(total_ecole_calcule),"kWh/an")
print("* Tentative accuracy\t",int(total_ecole/total_ecole_calcule*100),"%")